<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
#Start Here
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim.parsing.preprocessing import STOPWORDS

In [2]:
%pwd

'/home/adriann/Lambda/DS-Unit-4-Sprint-1-NLP/module4-topic-modeling'

In [3]:
%ls ../module1-text-data/data

Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv*
Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip
__MACOSX/
Untitled.ipynb
yelp_coffeeshop_review_data.csv


In [4]:
df = pd.read_csv('../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [5]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [6]:
STOPWORDS = set(STOPWORDS).union(set(['amazon']))

In [7]:
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [8]:
df['tokens'] = df['reviews.text'].apply(tokenize)

In [9]:
df['tokens']

0        [order, item, bad, quality, missing, backup, s...
1                   [bulk, expensive, way, products, like]
2                                 [duracell, price, happy]
3                  [work, brand, batteries, better, price]
4                 [batteries, long, lasting, price, great]
                               ...                        
28327    [got, yr, old, twins, yr, old, better, perfect...
28328    [bought, niece, christmas, gift, years, old, l...
28329    [nice, light, internet, browsing, keeping, ema...
28330    [tablet, absolutely, want, watch, tv, shows, m...
28331    [ninety, dollars, expectionations, low, good, ...
Name: tokens, Length: 28332, dtype: object

In [10]:
from collections import Counter

word_counts = Counter()

df['tokens'].apply(lambda x: word_counts.update(x))

word_counts.most_common(10)

[('great', 9464),
 ('batteries', 8148),
 ('tablet', 6685),
 ('good', 5906),
 ('price', 5221),
 ('use', 4484),
 ('love', 3549),
 ('bought', 3186),
 ('easy', 2954),
 ('kindle', 2891)]

In [11]:
id2word = corpora.Dictionary(df['tokens'])

In [12]:
id2word.token2id['tablet']

4195

In [13]:
type(id2word)

gensim.corpora.dictionary.Dictionary

In [14]:
id2word.doc2bow(tokenize("The tablet was not working when I powered it on"))

[(151, 1), (470, 1), (4195, 1)]

In [15]:
len(id2word.keys())

9620

In [16]:
id2word.filter_extremes(no_below=5, no_above=.95)

In [17]:
len(id2word.keys())

3580

In [18]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [19]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [20]:
lda = LdaMulticore(corpus=corpus,
                  id2word=id2word,
                  random_state=42,
                  num_topics=4,
                  passes=10,
                  workers=12)

In [21]:
lda.print_topics()

[(0,
  '0.045*"tablet" + 0.027*"kids" + 0.027*"great" + 0.025*"love" + 0.022*"easy" + 0.021*"use" + 0.020*"old" + 0.020*"games" + 0.018*"year" + 0.016*"loves"'),
 (1,
  '0.058*"great" + 0.044*"good" + 0.039*"price" + 0.030*"batteries" + 0.019*"tablet" + 0.015*"battery" + 0.014*"use" + 0.013*"work" + 0.012*"long" + 0.011*"product"'),
 (2,
  '0.021*"batteries" + 0.016*"tablet" + 0.011*"store" + 0.010*"apps" + 0.009*"use" + 0.009*"best" + 0.007*"like" + 0.007*"buy" + 0.007*"little" + 0.006*"need"'),
 (3,
  '0.033*"bought" + 0.027*"great" + 0.022*"tablet" + 0.021*"loves" + 0.021*"gift" + 0.018*"kindle" + 0.018*"use" + 0.017*"christmas" + 0.014*"purchased" + 0.014*"love"')]

In [22]:
import re
words = [re.findall(r'"(\w+)"',i[1]) for i in lda.print_topics()]

In [23]:
topics = [' '.join(i[0:5]) for i in words]

In [24]:
for id, i in enumerate(topics):
    print(f"--------Topic {id}--------")
    print(i, "\n")

--------Topic 0--------
tablet kids great love easy 

--------Topic 1--------
great good price batteries tablet 

--------Topic 2--------
batteries tablet store apps use 

--------Topic 3--------
bought great tablet loves gift 



In [25]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [26]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.126930  0.079141       1        1  29.652960
2      0.088636 -0.106800       2        1  28.774416
0     -0.137277 -0.026893       3        1  25.328138
3     -0.078289  0.054552       4        1  16.244493, topic_info=           Term         Freq        Total Category  logprob  loglift
21        great  8989.000000  8989.000000  Default  30.0000  30.0000
18    batteries  4652.000000  4652.000000  Default  29.0000  29.0000
191        kids  2504.000000  2504.000000  Default  28.0000  28.0000
27       bought  3188.000000  3188.000000  Default  27.0000  27.0000
33         good  5287.000000  5287.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1678     screen   331.022552  2059.533203   Topic4  -5.0039  -0.0106
95        works   301.911285  1533.803345   Topic4  -5.0960   0.1920
2085      books   278.605011  1453.191162   Topic4  -5.1763   0.1657
17        price   342.105438  4620.813477   Topic4  -4.9710  -0.7858
104        year   268.684784  1836.185669   Topic4  -5.2126  -0.1045

[354 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
990       1  0.017213  absolutely
990       2  0.075737  absolutely
990       3  0.165244  absolutely
990       4  0.743600  absolutely
2783      2  0.973790     adapter
...     ...       ...         ...
104       4  0.146499        year
960       1  0.023139       young
960       2  0.028923       young
960       3  0.873479       young
960       4  0.075200       young

[820 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4])

In [27]:
# def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    
#     coherence_values = []
    
#     for iter_ in range(passes):
#         for num_topics in range(start, limit, step):
#             model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
#             coherencemodel = CoherenceModel(model=model, dictionary=dictionary, corpus=corpus, coherence='u_mass')
#             coherence_values.append({'pass': iter_,
#                                     'num_topics': num_topics,
#                                     'coherence_score': coherencemodel.get_coherence()})
            
#     return coherence_values

In [28]:
# coherence_values = compute_coherence_values(dictionary=id2word,
#                                            corpus=corpus,
#                                            start=2,
#                                            limit=40,
#                                            step=2,
#                                            passes=10)

In [29]:
# topic_coherence = pd.DataFrame.from_records(coherence_values)

In [30]:
# topic_coherence

In [31]:
# topic_coherence.sort_values(by='coherence_score', ascending=False)

In [32]:
# import seaborn as sns
# ax = sns.lineplot(x='num_topics', y='coherence_score', data=topic_coherence)

In [33]:
distro = [lda[i] for i in corpus]

In [34]:
# list to hold dicts
new_distro = []
# grab all the predicted topics
for i in distro:
    # create a new dict that is labled from 0-14
    d_dist = {k:0 for k in range(0,15)}
    # grabs an indiviudal predicted topic
    for x in i:
        # Since a indivudal prediction is in a tuple x[0] is the topic number
        # and x[1] is the certancy of the prediction
        d_dist[x[0]] = x[1]
    # appned the dict to the list
    new_distro.append(d_dist)

In [35]:
pred_topics = pd.DataFrame.from_records(new_distro)

pred_topics['Primary_topics'] = pred_topics.idxmax(axis=1)

In [36]:
pred_topics

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Primary_topics
0,0.022994,0.025356,0.926471,0.025179,0,0,0,0,0,0,0,0,0,0,0,2
1,0.042429,0.043459,0.870510,0.043602,0,0,0,0,0,0,0,0,0,0,0,2
2,0.063517,0.799834,0.064435,0.072214,0,0,0,0,0,0,0,0,0,0,0,1
3,0.041897,0.872237,0.043036,0.042830,0,0,0,0,0,0,0,0,0,0,0,1
4,0.042046,0.873259,0.042581,0.042114,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,0.376969,0.023051,0.023277,0.576703,0,0,0,0,0,0,0,0,0,0,0,3
28328,0.032953,0.031387,0.031516,0.904144,0,0,0,0,0,0,0,0,0,0,0,3
28329,0.427432,0.555770,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,1
28330,0.534373,0.130494,0.321820,0.013312,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
df['Primary Topics'] = pred_topics['Primary_topics']

In [38]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tokens,Primary Topics
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, quality, missing, backup, s...",2
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expensive, way, products, like]",2
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...","[duracell, price, happy]",1
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...","[work, brand, batteries, better, price]",1
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...","[batteries, long, lasting, price, great]",1


In [39]:
rating_topic = df[['reviews.rating','Primary Topics']]

In [40]:
rating_topic

,reviews.rating,Primary Topics
0,3,2
1,4,2
2,5,1
3,5,1
4,5,1
...,...,...
28327,5,3
28328,4,3
28329,5,1
28330,5,0


In [41]:
rating_topic = rating_topic.groupby('Primary Topics').mean()

In [42]:
rating_topic.head(15)

,reviews.rating
Primary Topics,
0,4.665993
1,4.652556
2,3.876550
3,4.663455


## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling